In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [55]:
import os

stream = os.popen('curl -fsS https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip -o /tmp/trainDevTestTrees_PTB.zip')
output = stream.read()
output

''

In [56]:
stream = os.popen('unzip -q -o -d /tmp /tmp/trainDevTestTrees_PTB.zip')
output = stream.read()
output

''

In [57]:
stream = os.popen('rm -f /tmp/trainDevTestTrees_PTB.zip')
output = stream.read()
output

''

In [58]:
def loadsst(path):
    xs = []
    ys = []
    file1 = open(path, 'r')
    Lines = file1.readlines()
    for line in Lines:
        soup = line.split()
        ys.append(int(soup[0].lstrip('(')))
        tokens = []
        for chunk in soup[2:]:
            if not chunk.endswith(")"):
                continue
            tokens.append(chunk.rstrip(')'))
        xs.append(tokens)
    return xs, ys

ssttrainxs, ssttrainys = loadsst("/tmp/trees/train.txt")
sstvalidxs, sstvalidys = loadsst("/tmp/trees/dev.txt")
ssttestxs, ssttestys = loadsst("/tmp/trees/test.txt") 


In [59]:
for i in range(32):
    print(ssttrainys[i], " ".join(ssttrainxs[i]))


3 The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
4 The gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a column of words can not adequately describe co-writer\/director Peter Jackson 's expanded vision of J.R.R. Tolkien 's Middle-earth .
3 Singer\/composer Bryan Adams contributes a slew of songs -- a few potential hits , a few more simply intrusive to the story -- but the whole package certainly captures the intended , er , spirit of the piece .
2 You 'd think by now America would have had enough of plucky British eccentrics with hearts of gold .
3 Yet the act is still charming here .
4 Whether or not you 're enlightened by any of Derrida 's lectures on `` the other '' and `` the self , '' Derrida is an undeniably fascinating and playful fellow .
4 Just the labour involved in creating the layered richness of the 

In [60]:
len( ssttrainxs)

8544

In [61]:
character_set = set(" ".join([" ".join(ssttrainxs[i]) for i in range(len(ssttrainxs)) ]))

In [62]:
len(character_set)

93

In [63]:
int_to_char = {}
char_to_int = {}
c = 0
for i in character_set:
  int_to_char[c] = i
  char_to_int[i] = c
  c = c+1
print(int_to_char)
print(char_to_int)

{0: '?', 1: 'M', 2: 'c', 3: '!', 4: '*', 5: 'i', 6: 'D', 7: 'K', 8: 'a', 9: 'm', 10: 'u', 11: 'C', 12: '3', 13: '4', 14: 'ñ', 15: 'é', 16: '&', 17: '=', 18: ':', 19: 'n', 20: 'S', 21: 'Y', 22: 'W', 23: 'A', 24: '#', 25: '6', 26: 'j', 27: 'X', 28: 'ô', 29: 'è', 30: '9', 31: 'g', 32: 'f', 33: '0', 34: ';', 35: 'V', 36: 'p', 37: 'N', 38: 'ï', 39: 's', 40: 'h', 41: 'd', 42: 'l', 43: 'P', 44: '-', 45: 'o', 46: '2', 47: 'T', 48: 'G', 49: '7', 50: 'x', 51: 'â', 52: '+', 53: 'y', 54: '.', 55: 'í', 56: 'R', 57: 'U', 58: 'J', 59: '/', 60: 'w', 61: 'L', 62: "'", 63: 'ó', 64: 'k', 65: 'á', 66: '8', 67: 't', 68: 'ç', 69: ' ', 70: 'B', 71: 'b', 72: 'I', 73: 'q', 74: '$', 75: 'E', 76: 'ã', 77: 'v', 78: 'O', 79: 'F', 80: '1', 81: 'æ', 82: 'z', 83: 'r', 84: 'Q', 85: '5', 86: 'ü', 87: 'H', 88: 'e', 89: 'Z', 90: ',', 91: '`', 92: '\\'}
{'?': 0, 'M': 1, 'c': 2, '!': 3, '*': 4, 'i': 5, 'D': 6, 'K': 7, 'a': 8, 'm': 9, 'u': 10, 'C': 11, '3': 12, '4': 13, 'ñ': 14, 'é': 15, '&': 16, '=': 17, ':': 18, 'n': 19, 

In [64]:
#creating a dataset of first n characters predicting n+1 th character.
n =100
train_list = [" ".join(ssttrainxs[i]) for i in range(len(ssttrainxs))] 
train_data = []
y_train = []
for sentence in train_list:
  for i in range(0,len(sentence)-n):
    train_data.append(sentence[i:i+n])
    y_train.append(sentence[i+n])

In [65]:
for i in range(50):
  print(train_data[i]," ",y_train[i])

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash    
he Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash    e
e Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash e   v
 Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash ev   e
Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash eve   n
ock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even    
ck is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even    g
k is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even g   r
 is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even gr   e
is destined to be the 21st Century 's new `` Conan '' a

In [66]:
def convert_char_to_int(train_data,y,char_to_int_dict):
  train_x = []
  train_y = []
  for i in range(len(train_data)):
    d = []
    for j in train_data[i]:
      num = char_to_int_dict[j]
      d.append(num)
    train_x.append(d)
    train_y.append(char_to_int_dict[y[i]])
  return train_x,train_y

In [67]:
train_x,train_y = convert_char_to_int(train_data,y_train,char_to_int)

In [68]:
import random
temp = list(zip(train_x,train_y))
random.shuffle(temp)
train_x,train_y = zip(*temp)
print(train_x[0:2],train_y[0:2])

([83, 53, 69, 42, 5, 67, 67, 42, 88, 69, 40, 8, 42, 32, 44, 40, 45, 10, 83, 69, 90, 69, 8, 32, 67, 88, 83, 44, 39, 2, 40, 45, 45, 42, 69, 39, 36, 88, 2, 5, 8, 42, 69, 8, 71, 45, 10, 67, 69, 5, 19, 67, 88, 83, 32, 8, 5, 67, 40, 69, 10, 19, 41, 88, 83, 39, 67, 8, 19, 41, 5, 19, 31, 69, 90, 69, 39, 67, 83, 88, 67, 2, 40, 88, 41, 69, 45, 10, 67, 69, 67, 45, 69, 30, 33, 69, 9, 5, 19, 10], [67, 69, 8, 71, 45, 10, 67, 69, 8, 69, 32, 5, 42, 9, 9, 8, 64, 88, 83, 69, 60, 40, 88, 19, 69, 40, 88, 69, 2, 8, 19, 69, 71, 88, 69, 60, 8, 2, 64, 53, 69, 60, 5, 67, 40, 45, 10, 67, 69, 2, 42, 45, 71, 71, 88, 83, 5, 19, 31, 69, 67, 40, 88, 69, 8, 10, 41, 5, 88, 19, 2, 88, 69, 45, 77, 88, 83, 69, 67, 40, 88, 69, 40, 88, 8, 41, 69, 8, 19, 41, 69, 39, 67, 5, 42, 42, 69, 9, 8, 5]) (67, 19)


In [47]:
#normalizing them in the range of 0-1
train_x_2 = [[[i/93] for i in train_x[j]] for j in range(len(train_x))]
train_x_2[0]

TypeError: ignored

In [120]:
import torch
from torch import nn
import argparse
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader



class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 7
        self.num_layers = 1
        self.input_size = n

        n_vocab = len(character_set)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        #print(x.shape,embed.shape)
        output, state = self.lstm(embed, prev_state)
        #print("output",output.shape)
        logits = self.fc(output[:,-1,:])
        #print("logits", logits.shape)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))
 

In [104]:

from torch.utils.data import DataLoader
def train(dataset, model, batch_size,max_epochs,sequence_length):
    model.train()

    dataloader = DataLoader(dataset, batch_size=batch_size)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(max_epochs):
        state_h, state_c = model.init_state(sequence_length)

        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()
            print("x",x.shape)
            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            #print("y_pred",y_pred.shape)
            #print("y",y.shape)
            loss = loss_fn(y_pred, y)
            probs = torch.nn.functional.softmax(y_pred, dim=-1).detach().numpy()
            preds = np.argmax(probs,axis =1)
            acc = np.sum(preds==y.numpy())/(preds.shape[0])
            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item(),'batch_acc':acc })

In [78]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,train_x,train_y,character_set):
        self.train_x = train_x
        self.train_y = train_y
        self.character_set = character_set

    def __len__(self):
        return len(self.train_x) 

    def __getitem__(self, index):
        y_val = train_y[index]
        y_ohe = [0]*len(character_set)
        y_ohe[y_val] = 1
        return (
            torch.tensor(self.train_x[index]),
            torch.tensor(train_y[index]),
        )

In [106]:

training_data = Dataset(train_x,train_y,character_set)
model = Model(training_data)
train(training_data, model, batch_size = 512 , max_epochs = 5,sequence_length = 100)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Streaming output truncated to the last 5000 lines.
x torch.Size([512, 100])
torch.Size([512, 100]) torch.Size([512, 100, 7])
y_pred torch.Size([512, 93])
y torch.Size([512])
{'epoch': 2, 'batch': 122, 'loss': 2.448197841644287, 'batch_acc': 0.244140625}
x torch.Size([512, 100])
torch.Size([512, 100]) torch.Size([512, 100, 7])
y_pred torch.Size([512, 93])
y torch.Size([512])
{'epoch': 2, 'batch': 123, 'loss': 2.4225103855133057, 'batch_acc': 0.23828125}
x torch.Size([512, 100])
torch.Size([512, 100]) torch.Size([512, 100, 7])
y_pred torch.Size([512, 93])
y torch.Size([512])
{'epoch': 2, 'batch': 124, 'loss': 2.460294723510742, 'batch_acc': 0.23828125}
x torch.Size([512, 100])
torch.Size([512, 100]) torch.Size([512, 100, 7])
y_pred torch.Size([512, 93])
y torch.Size([512])
{'epoch': 2, 'batch': 125, 'loss': 2.488246440887451, 'batch_acc': 0.251953125}
x torch.Size([512, 100])
torch.Size([512, 100]) torch.Size([512, 100, 7])
y_pred torch.Size([512, 93])
y torch.Size([512])
{'epoch': 2, 'b

In [44]:
training_data = Dataset(train_x,train_y,character_set)
model = Model(training_data)
train(training_data, model, batch_size = 256 , max_epochs = 5,sequence_length = 100)

RuntimeError: ignored

In [122]:
def predict(model, text, next_chars=100,topk=3):
    model.eval()

    
    
    state_h, state_c = model.init_state(len(text))
    for i in range(0, next_chars):
        
        x = torch.tensor([[char_to_int[j] for j in text[i:]]])

        y_pred, (state_h, state_c) = model(x, (state_h, state_c))
        print(y_pred.shape)
        last_char_logits = y_pred[0]
        p = torch.nn.functional.softmax(last_char_logits, dim=-1).detach().numpy()
        #print(p.shape)
        ind = np.argpartition(p, -1*topk)[-1*topk:]
        char_index = np.random.choice(ind)
        #char_index = np.argmax(p)
        text = text+int_to_char[char_index]

    return text

In [51]:
model

Model(
  (embedding): Embedding(94, 7)
  (lstm): LSTM(7, 128, dropout=0.2)
  (fc): Linear(in_features=128, out_features=93, bias=True)
)

In [123]:
test_text = "i'm learning data science  "
complete_text = predict(model,test_text,1000)
complete_text

torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([1, 93])
torch.Size([1, 27]) torch.Size([1, 27, 7])
torch.Size([

"i'm learning data science  or thatherallicen ang ther and .ser titharan .. an t . ticthing an .t . tind ande al t tichender . .titit an t .sthingendinde .thit ar at . t .st an atit ... . t . an ... atitiche ..hat atitharan ..handit t . an aler aralerall tharathatit t t atitindithicat thatind and .sen anderat .st ...handichenghen . .tin tin anditit ..han the ande t .se t are ale .. tichanghe t an .s .. al ... thangen at t tind . tinge t theral . thang atithan . .. thend t an alllicer aratititheratitind .se arer ang tharall tin . athar tich thathatitictingh an an andichange angharaler athareran atin ... ..here t at alendin thinge ..... at . ...h thicat ting . thin an . . arer ..hande thind .ser ticthander an atin . .. tin titit . .t an theraticeresticathathande at .se thande .. tichen ale thicen . therall ang thinding athicall ar t t ander ticthithitit .s ..hat .s tin aler atin alll tin the ..h andendinder .ser atherale t t an alllit t an . angh tithinghe ..h the t atin andithathanghe t

In [56]:
complete_text

"The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash  the t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t "

In [1]:
model.eval()

NameError: ignored

In [48]:
train(training_data, model, batch_size = 1024 , max_epochs = 5,sequence_length = 10)

{'epoch': 0, 'batch': 0, 'loss': 0.8239567279815674}
{'epoch': 0, 'batch': 1, 'loss': 0.733441174030304}
{'epoch': 0, 'batch': 2, 'loss': 0.39701682329177856}
{'epoch': 0, 'batch': 3, 'loss': 0.17971742153167725}
{'epoch': 0, 'batch': 4, 'loss': 0.1376793533563614}
{'epoch': 0, 'batch': 5, 'loss': 0.12338676303625107}
{'epoch': 0, 'batch': 6, 'loss': 0.07797113060951233}
{'epoch': 0, 'batch': 7, 'loss': 0.07060865312814713}
{'epoch': 0, 'batch': 8, 'loss': 0.0710889920592308}
{'epoch': 0, 'batch': 9, 'loss': 0.07016865164041519}
{'epoch': 0, 'batch': 10, 'loss': 0.19844573736190796}
{'epoch': 0, 'batch': 11, 'loss': 0.3606850504875183}
{'epoch': 0, 'batch': 12, 'loss': 0.427432656288147}
{'epoch': 0, 'batch': 13, 'loss': 0.3637229800224304}
{'epoch': 0, 'batch': 14, 'loss': 0.34360137581825256}
{'epoch': 0, 'batch': 15, 'loss': 0.3339104950428009}
{'epoch': 0, 'batch': 16, 'loss': 0.19630001485347748}
{'epoch': 0, 'batch': 17, 'loss': 0.08146701008081436}
{'epoch': 0, 'batch': 18, 'los

KeyboardInterrupt: ignored

In [118]:
a = torch.tensor([[1,2,3,4],[5,6,7,8]])
s = torch.nn.functional.softmax(a,dim=0)
print(s)

RuntimeError: ignored